In [96]:
import spacy
import pandas as pd
import numpy as np


In [57]:
text_data = pd.read_csv('clinical_trials_dataset.csv')
#text_data = text_data.dropna(axis=0,how='any').reset_index(drop=True)
text_data = text_data[['OfficialTitle','StudyType','StartDate',
           'Condition','ConditionAncestorTerm',
           'ArmGroupInterventionName','InterventionArmGroupLabel','InterventionDescription']].copy()
#text_data[text_data['ArmGroupInterventionName'].str.contains('Genetic')]
text_data.isna().sum()

OfficialTitle                  53
StudyType                       0
StartDate                      42
Condition                       0
ConditionAncestorTerm         571
ArmGroupInterventionName     1128
InterventionArmGroupLabel    1124
InterventionDescription      1158
dtype: int64

In [110]:
#deal with nan values column by column
test = text_data.copy()
test = test.fillna({'StartDate':'unknown'}).copy()
test = test.fillna({'OfficialTitle':'unknown'}).copy()
test['ConditionAncestorTerm'] = test['ConditionAncestorTerm'].fillna(test['Condition'])
test = test.fillna({'ArmGroupInterventionName':'other or NA'}).copy()
test = test.fillna({'InterventionArmGroupLabel':'other or NA'}).copy()
test = test.fillna({'InterventionDescription':'other or NA'}).copy()
text_data = test.copy()

del test

In [112]:
#replace nan values with other appropriate values
text_data.isna().sum()

OfficialTitle                0
StudyType                    0
StartDate                    0
Condition                    0
ConditionAncestorTerm        0
ArmGroupInterventionName     0
InterventionArmGroupLabel    0
InterventionDescription      0
dtype: int64

In [119]:
text_data[text_data['ArmGroupInterventionName'].str.contains('Gene',case=False)]

,OfficialTitle,StudyType,StartDate,Condition,ConditionAncestorTerm,ArmGroupInterventionName,InterventionArmGroupLabel,InterventionDescription
0,"Phase I/IIa, First-in-human, Open-label, Singl...",Interventional,September 2024,Drug Resistant Epilepsy,Brain Diseases|Central Nervous System Diseases...,Genetic: lentiviral gene therapy,lentiviral gene therapy treatment (Interventio...,lentiviral gene therapy to treat drug resistan...
1,"Growth and Development, Health-related Quality...",Observational,"June 5, 2023",Transfusion-dependent Beta-Thalassemia|Gene Th...,"Anemia, Hemolytic, Congenital|Anemia, Hemolyti...",Genetic: Gene therapy,Gene therapy group,Autologous edited hematopoietic stem cell tran...
2,Phase I Trial of Atezolizumab and Interleukin-...,Interventional,"October 14, 2021",Non-small Cell Lung Cancer,Respiratory Tract Neoplasms|Thoracic Neoplasms...,Combination Product: Atezolizumab and Interleu...,Atezolizumab and Interleukin-12 Gene Therapy,To determine the safety and tolerability of th...
3,Gene Therapy for X Linked Severe Combined Immu...,Interventional,"May 1, 2020",Gene Therapy,Immune System Diseases|Primary Immunodeficienc...,Device: Lentiviral Vector Gene Therapy,Experimental Group,Lentiviral vector to transfer IL2RG complement...
4,Gene Therapy Follow up Protocol for Subjects P...,Observational,"May 8, 2020",Cervical Intraepithelial Neoplasia|Cervical Ne...,Precancerous Conditions|Uterine Cervical Disea...,Drug: GeneTherapy,Cohort 1,Observation/ Gene Therapy Long-term Follow-up....
...,...,...,...,...,...,...,...,...
6036,"A Phase 1 Study of Ipilumumab, Nivolumab and T...",Interventional,"July 10, 2019",Anatomic Stage 0 Breast Cancer AJCC v8|Anatomi...,"Neoplasms, Glandular and Epithelial|Neoplasms ...",Biological: Ipilimumab|Biological: Nivolumab|B...,"Treatment (talimogene laherparepvec, nivolumab...",Given IV|Given IV|Given intratumorally
6039,Identifying Maturity-onset Diabetes of the You...,Interventional,March 2024,Maturity-onset Diabetes of the Young|Monogenic...,Glucose Metabolism Disorders|Metabolic Disease...,Diagnostic Test: Genetic testing for 4 MODY ge...,patients labelled as Type 1 or Type 2 Diabetes,"Genetic testing for 4 MODY genes (GCK, HNF1A, ..."
6076,A Phase I/II Trial of Temozolomide and ABT-888...,Interventional,"July 13, 2009",Brain and Central Nervous System Tumors,"Astrocytoma|Glioma|Neoplasms, Neuroepithelial|...",Drug: temozolomide|Drug: veliparib|Genetic: DN...,Dose Escalation|Dose Escalation|Dose Escalatio...,other or NA
6105,A Phase 2 Study of Talimogene Laherparepvec (T...,Interventional,"December 31, 2019",Leiomyosarcoma|Liposarcoma|Sarcoma G2|Sarcoma ...,"Neoplasms, Connective and Soft Tissue|Neoplasm...",Procedure: Biopsy|Procedure: Biospecimen Colle...,"Treatment (talimogene laherparepvec, radiation...",Undergo tumor biopsy|Undergo collection of blo...


In [ ]:
s